In [ ]:
pip install -r requirements.txt --quiet

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
pip install tesseract

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
pip install label_studio_sdk

In [ ]:
from label_studio_sdk import Client
from label_studio_sdk import Project
from label_studio_sdk.data_manager import Column, Filters, Operator, Type
from datetime import datetime
import json 
from IPython.display import Image as displayImage
from PIL import Image
import os
from pytesseract import pytesseract

pytesseract.run_tesseract("lal3.png","output_hocr",extension='png', lang=None,config="--psm 4 -c tessedit_create_hocr=1")


In [ ]:
LABEL_STUDIO_URL = 'https://slick-poems-type-122-161-53-190.loca.lt'
API_KEY = 'd6b723213a1a324b1a76dc4a0f2c6787aabd575c'
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()


{'status': 'UP'}

In [ ]:
projects=ls.get_projects()
print("Total Projects present: "+str(len(projects)))
print("Select a project from below:")
for idx,p in enumerate(projects):
    print(str(idx+1)+'.'+p.title)
selProjectNum=int(input("Select a project..."))-1

project=projects[selProjectNum]

Total Projects present: 1
Select a project from below:
1.LalReportTry
Select a project...1


In [ ]:
filters = Filters.create(Filters.OR, [
    Filters.item(
        Column.completed_at,
        Operator.IN,
        Type.Datetime,
        Filters.value(
            datetime(2021, 11, 1),
            datetime.now()
        )
    )
])
tasks = project.get_tasks(filters=filters,ordering=[Column.completed_at])
print(tasks)



[{'id': 2, 'data': {'imagepath': '/data/upload/4/f784084c-Covid_Sample_Report1_Lal_P_2.png'}, 'meta': {}, 'created_at': '2022-05-07T15:30:25.824247Z', 'updated_at': '2022-05-07T15:33:37.437711Z', 'is_labeled': True, 'overlap': 1, 'project': 4, 'file_upload': 3, 'annotations': [{'id': 5, 'created_username': ' vatsalgoyal44@gmail.com, 1', 'created_ago': '1\xa0minute', 'result': [{'original_width': 632, 'original_height': 888, 'image_rotation': 0, 'value': {'x': 5.063291139240507, 'y': 15.427927927927929, 'width': 34.810126582278485, 'height': 3.0405405405405403, 'rotation': 0}, 'id': 'nybzdgbRcH', 'from_name': 'bbox', 'to_name': 'image', 'type': 'rectangle', 'origin': 'manual'}, {'original_width': 632, 'original_height': 888, 'image_rotation': 0, 'value': {'x': 5.063291139240507, 'y': 15.427927927927929, 'width': 34.810126582278485, 'height': 3.0405405405405403, 'rotation': 0, 'labels': ['Name']}, 'id': 'nybzdgbRcH', 'from_name': 'label', 'to_name': 'image', 'type': 'labels', 'origin': '

In [ ]:
annotation=tasks[-1]['annotations'][-1]

#get the labels and bbox
result=annotation['result']

labels=[d for d in result if d['type'] in ['labels']]

#read the labels.json
userDefinedLabels=None

with open("data/labels.json", "r") as jsonfile:
    userDefinedLabels=json.load(jsonfile)

if userDefinedLabels==None:
    print("Error! No User defined label present!")
    sys.exit()

for ulabel in userDefinedLabels:
    #assuming there is only one
    print(ulabel)
    uvalue=[item['value'] for item in labels if item['value']['labels'][0] == ulabel['labelName']][0]
    ulabel['x']=uvalue['x']
    ulabel['y']=uvalue['y']
    ulabel['width']=uvalue['width']
    ulabel['height']=uvalue['height']
    
with open("data/labels.json", "w") as outfile:
    json.dump(userDefinedLabels, outfile)


{'labelName': 'Name', 'labelType': 'Person'}
{'labelName': 'Collection Date', 'labelType': 'Date'}
{'labelName': 'Result', 'labelType': 'CovidResult'}


In [ ]:
import bs4
img = Image.open("lal3.png")
width, height = img.size
xml_input = open("output_hocr.hocr","r",encoding="utf-8")
soup = bs4.BeautifulSoup(xml_input,'lxml')
ocr_lines = soup.findAll("span", {"class": "ocr_line"})
#We will save coordinates of line and the text contained in the line in lines_structure list
lines_structure = []
for line in ocr_lines:
    line_text = line.text.replace("\n"," ").strip()
    title = line['title']
    #The coordinates of the bounding box
    x1,y1,x2,y2 = map(int, title[5:title.find(";")].split())
    lines_structure.append({"x1":(x1*100.0)/width,"y1":(y1*100.0)/height,"x2":(x2*100.0)/width,"y2":(y2*100.0)/height,"text": line_text})

In [ ]:
textextracted={}
for ulabel in userDefinedLabels:
  print(ulabel['labelName'])
  textextracted[ulabel['labelName']] = ""
  for struct in lines_structure:
    intersectingwidth = min(struct['x2'],ulabel['x']+ulabel['width'])-max(struct['x1'],ulabel['x'])
    intersectingheigth = min(struct['y2'],ulabel['y']+ulabel['height'])-max(struct['y1'],ulabel['y'])
    if intersectingheigth<=0 or intersectingwidth<=0:
      intersectingarea = 0
    else:
      intersectingarea=abs(intersectingheigth*intersectingwidth)
    boxarea = abs((struct['x1']-struct['x2'])*(struct['y1']-struct['y2']))
    labelarea = ulabel['width']*ulabel['height']
    overlap = intersectingarea/(boxarea*1.0)
    overlap2 = intersectingarea/(labelarea*1.0)
    if overlap >= 0.9 or overlap2 >= 0.9:
      textextracted[ulabel['labelName']] = textextracted[ulabel['labelName']] + " "+struct['text']
  print(textextracted[ulabel['labelName']])  
  


Name

Collection Date
 — casi
Result
 tne se  ‘SARS.CoV-2(COVID-19) QUALITATIVE REAL TIME RT-PCR  “Type of Specimen Nasopharyngeal COropharynges! Swabs Result Negative


In [ ]:
!python -m spacy download en_core_web_md


     |████████████████████████████████| 96.4 MB 985 kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import spacy
from collections import defaultdict

nlp = spacy.load('en_core_web_md')


In [ ]:
keys=[]
values=[]
for ulabel in userDefinedLabels:
    if ulabel['labelName']=='Name':
      text1 = nlp(textextracted[ulabel['labelName']])
      for w in text1.ents:
        if w.label_=='PERSON':
          keys.append('PERSON')
          values.append(w.text)
          print(w.text)
    elif ulabel['labelName']=='Date':
      text1 = nlp(textextracted[ulabel['labelName']])
      for w in text1.ents:
        if w.label_=='DATE':
          keys.append('DATE')
          values.append(w.text)
          print(w.text)
    elif ulabel['labelName']=='Result':
      text1 = textextracted[ulabel['labelName']]
      text1 = text1.lower()
      if text1.find('positive')>=0:
        keys.append('RESULT')
        values.append('Positive')
        print('Positive')
      if text1.find('negative')>=0:
        keys.append('RESULT')
        values.append('Negative')
        print('Negative')

Negative
